In [1]:
# Local directory where the Data are mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

In [2]:
skip_subjects = ["ANGELONI_GIUSEPPINA", "GATTAMORTA_NATALINA", "IONTA_LUCIANA"]

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("datasets/dataset-survivor.csv")

In [5]:
import os
import numpy as np
import imageio
import sys

from nilearn.image import resample_to_img

from gliomi import *

def generate_dataset(sequence, percentile, template_subject, dataset, skip_subjects):
    
    print(f"Generating dataset for {sequence} with percentile {percentile}.")
    print(f"We are using {template_subject} as template.")
    print(f"We are skipping {skip_subjects}")

    mask = "T2ROI"
    dataset_path = f"dataset-{sequence}-{percentile}"

    # sequence for template subject
    orig = Subject(datasetDir, template_subject)
    t1_orig = orig.get_sequence(sequence)

    for i in range(dataset.shape[0]):

        subject = dataset.iloc[i, 1]

        if subject in skip_subjects:
            next

        label = df.iloc[i, 2]

        try:

            s = Subject(datasetDir, subject)

            dst = f"{dataset_path}/{label}/{subject}"

            os.makedirs(dst, exist_ok=True)

            t1 = s.get_sequence(sequence)
            t1_reshaped = resample_to_img(t1, t1_orig, interpolation='nearest')

            roi = s.get_roi(mask)
            roi_shaped = resample_to_img(roi, t1_orig, interpolation='nearest')
            
            roi_shaped_data = roi_shaped.get_fdata()
            roi_sizes = np.sum(roi_shaped_data, axis=(0, 1))
            non_empty_sizes = roi_sizes[np.where(roi_sizes > 0)]
            percentile_val = np.percentile(non_empty_sizes, percentile)
            roi_indexes = np.where(roi_sizes > percentile_val)[0]

            # Getting resampled subject data
            # img = t1_reshaped.dataobj
            
            img = np.asarray(t1_reshaped.dataobj)
            img = np.uint8(img / np.max(img) * 255)

            # Save PNG for each slice with mask size above the selected percentile
            for z in roi_indexes:
                imageio.imwrite(f"{dst}/{z}.png", img[:,:,z])

        except FileNotFoundError as e:
            print(e)
        except TypeError as e:
            print(e)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print(f"Skipping subject {subject}: missing sequence {sequence}")

ModuleNotFoundError: No module named 'imageio'

In [6]:
def generate_dataset_not_tumor(sequence, template_subject, dataset, skip_subjects):
    
    print(f"We are using {template_subject} as template.")
    print(f"We are skipping {skip_subjects}")

    mask = "T2ROI"
    dataset_path = f"dataset-{sequence}-not-tumor"

    # sequence for template subject
    orig = Subject(datasetDir, template_subject)
    t1_orig = orig.get_sequence(sequence)

    for i in range(dataset.shape[0]):

        subject = dataset.iloc[i, 1]

        if subject in skip_subjects:
            next

        label = df.iloc[i, 2]

        try:

            s = Subject(datasetDir, subject)

            dst = f"{dataset_path}/{label}/{subject}"

            os.makedirs(dst, exist_ok=True)

            t1 = s.get_sequence(sequence)
            t1_reshaped = resample_to_img(t1, t1_orig, interpolation='nearest')

            roi = s.get_roi(mask)
            roi_shaped = resample_to_img(roi, t1_orig, interpolation='nearest')
            
            roi_shaped_data = roi_shaped.get_fdata()
            roi_sizes = np.sum(roi_shaped_data, axis=(0, 1))
            roi_indexes = np.where(roi_sizes == 0)[0]

            # Getting resampled subject data
            # img = t1_reshaped.dataobj
            
            img = np.asarray(t1_reshaped.dataobj)
            img = np.uint8(img / np.max(img) * 255)

            # Save PNG for each slice with mask size above the selected percentile
            for z in roi_indexes:
                imageio.imwrite(f"{dst}/{z}.png", img[:,:,z])

        except FileNotFoundError as e:
            print(e)
        except TypeError as e:
            print(e)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print(f"Skipping subject {subject}: missing sequence {sequence}")

In [7]:
generate_dataset_not_tumor("FLAIR", "ALESSANDRINI_GLAUCO", df, skip_subjects)

We are using ALESSANDRINI_GLAUCO as template.
We are skipping ['ANGELONI_GIUSEPPINA', 'GATTAMORTA_NATALINA', 'IONTA_LUCIANA']
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BIANCHI_S/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BOVE_A/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/LUPO_ASSUNTA/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MGMT_Labels.csv/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MONTI_E/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/PELUSO_A/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/result-scalati.txt/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENT

In [12]:
generate_dataset("rCBV", 70, "ALESSANDRINI_GLAUCO", df, skip_subjects)

Generating dataset for rCBV with percentile 70.
We are using ALESSANDRINI_GLAUCO as template.
We are skipping ['ANGELONI_GIUSEPPINA', 'GATTAMORTA_NATALINA', 'IONTA_LUCIANA']
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/ANGELONI_GIUSEPPINA/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/ASTA_G/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BAGNOLI_VINCENZO/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BALZAMO_A/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BATTISTA_DOMENICA/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BIANCHI_GIOVANNI/rCBV_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BIANCHI_ORAZIO/rCBV_registered.nii'
N

In [6]:
for sequence in ["T1", "ADC", "FLAIR"]:
    for percentile in [70, 50, 30]:
        generate_dataset(sequence, percentile, "ALESSANDRINI_GLAUCO", df, skip_subjects)

No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/ASTA_G/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BALZAMO_A/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BIANCHI_S/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BOVE_A/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/CELLINI_T/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/CHERRI_M/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/COLAMARTINI_GIUSEPPINA/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/COLETTA_MARIA/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/COSIMI_MASSIMO/T1_re

No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/ISMAIL_A/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/LUPO_ASSUNTA/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MARCONI_E/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MARTELLA_COSIMO/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MGMT_Labels.csv/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MONTI_E/T1_registered.nii'
Unexpected error: <class 'IndexError'>
Skipping subject PAGLIAROLI_LUCIA: missing sequence T1
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/PASCARELLA_P/T1_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/PELUSO_A/T1_registered.nii'
No s

Unexpected error: <class 'IndexError'>
Skipping subject PAGLIAROLI_LUCIA: missing sequence ADC
Unexpected error: <class 'IndexError'>
Skipping subject PROIETTI_GIOVANNI: missing sequence ADC
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/result-scalati.txt/ADC_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/result.txt/ADC_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/RE_ATTILIO/ROI/T2ROI.nii'
Unexpected error: <class 'IndexError'>
Skipping subject RUSCITO_ELISABETTA: missing sequence ADC
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/ZANATTA_CARLO/ADC_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BIANCHI_S/FLAIR_registered.nii'
No such file or no access: '/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/BOVE_A/FLAIR_registered.nii'
No such file or no access: '/

In [ ]:
'''BLOCCO COPIA FATTO DA EMANUELA'''

index_array=[i for i in range(df.shape[0])]
np.random.shuffle(index_array)
#print(index_array)
train_sub=[i for i in index_array[:120]]
val_sub=[i for i in index_array[121:]]

import os
import numpy as np
import imageio
import Subject
import sys

from nilearn.image import resample_to_img

sequence = "T1"
mask = "T2ROI"
percentile = 70
dataset_path = f"dataset-{percentile}-all"

use_all = False

orig = Subject.Subject(datasetDir, "ALESSANDRINI_GLAUCO")
t1_orig = orig.get_sequence(sequence)

for i in index_array[:120]:

    subject = df.iloc[i, 1]
    
    if subject in skip_subjects:
        next

    label = df.iloc[i, 2]
    
    try:

        s = Subject.Subject(datasetDir, subject)

        dst = f"{dataset_path}/{label}/{subject}"

        os.makedirs(dst, exist_ok=True)
        
        t1 = s.get_sequence(sequence)
        
        t1_reshaped = resample_to_img(t1, t1_orig, interpolation='nearest')
        
        if not use_all:
            roi = s.get_roi(mask)
            roi_shaped = resample_to_img(roi, t1_orig, interpolation='nearest')
            roi_shaped_data = roi_shaped.get_fdata()
            roi_sizes = np.sum(roi_shaped_data, axis=(0, 1))
            non_empty_sizes = roi_sizes[np.where(roi_sizes > 0)]
            percentile_val = np.percentile(non_empty_sizes, percentile)
            roi_indexes = np.where(roi_sizes > percentile_val)[0]

        img = t1_reshaped.dataobj
        
        print(img.shape)

        if use_all:
            roi_indexes = list(range(img.shape[2]))

        for z in roi_indexes:
            imageio.imwrite(f"{dst}/{sequence}-{z}.png", img[:,:,z])
           
    except FileNotFoundError as e:
        print(e)
    except TypeError as e:
        print(e)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        print(f"Skipping subject {subject}: missing sequence {sequence}")

In [ ]:
import os
import numpy as np
import imageio
import Subject
import sys

from nilearn.image import resample_to_img

sequence = "T1"
mask = "T2ROI"
percentile = 70
dataset_path = f"dataset-{percentile}-all"

use_all = False

orig = Subject.Subject(datasetDir, "ALESSANDRINI_GLAUCO")
t1_orig = orig.get_sequence(sequence)

for i in range(df.shape[0]):

    subject = df.iloc[i, 1]
    
    if subject in skip_subjects:
        next

    label = df.iloc[i, 2]
    
    try:

        s = Subject.Subject(datasetDir, subject)

        dst = f"{dataset_path}/{label}/{subject}"

        os.makedirs(dst, exist_ok=True)
        
        t1 = s.get_sequence(sequence)
        
        t1_reshaped = resample_to_img(t1, t1_orig, interpolation='nearest')
        
        if not use_all:
            roi = s.get_roi(mask)
            roi_shaped = resample_to_img(roi, t1_orig, interpolation='nearest')
            roi_shaped_data = roi_shaped.get_fdata()
            roi_sizes = np.sum(roi_shaped_data, axis=(0, 1))
            non_empty_sizes = roi_sizes[np.where(roi_sizes > 0)]
            percentile_val = np.percentile(non_empty_sizes, percentile)
            roi_indexes = np.where(roi_sizes > percentile_val)[0]

        img = t1_reshaped.dataobj
        
        print(img.shape)

        if use_all:
            roi_indexes = list(range(img.shape[2]))

        for z in roi_indexes:
            imageio.imwrite(f"{dst}/{sequence}-{z}.png", img[:,:,z])
           
    except FileNotFoundError as e:
        print(e)
    except TypeError as e:
        print(e)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        print(f"Skipping subject {subject}: missing sequence {sequence}")

In [ ]:
# Plot a grid of above 70% masked data (data + mask)

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

indexes = s.get_roi_index_percentile("SOLID", 2, 70)

img = t1.dataobj
roi = s.get_roi("SOLID").dataobj
img_roi = np.asarray(img) * np.asarray(roi)

fig = plt.figure(figsize=(40., 40.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(5, 4),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for ax, im, im_roi in zip(grid, [img[:,:,z_slice] for z_slice in indexes], [img_roi[:,:,z_slice] for z_slice in indexes]):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)
    ax.imshow(im_roi, alpha=0.5)

plt.show()

In [ ]:
# Plot only masks

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

indexes = s.get_roi_index_percentile("SOLID", 2, 70)

img = t1.dataobj
roi = s.get_roi("SOLID").dataobj

img = np.asarray(img) * np.asarray(roi)

fig = plt.figure(figsize=(40., 40.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(5, 4),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )

for ax, im in zip(grid, [img[:,:,z_slice] for z_slice in indexes]):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)

plt.show()

In [ ]:
# Return bounding box of roi

s.bounding_box("SOLID")

In [ ]:
from PIL import Image

In [ ]:
data = np.asarray(t1.dataobj)

In [ ]:
im = Image.fromarray(np.uint8(data[:,:,78] / np.max(data) * 255))

In [ ]:
im